In [1]:
!pip install numpy scipy pandas scikit-learn pyhrv wfdb torch torchvision tqdm kagglehub
!pip install kagglehub torch numpy scipy scikit-learn
!pip install peakutils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 kB 40.8 MB/s eta 0:00:00
  Created wheel for spectrum: filename=spectrum-0.9.0-cp312-cp312-linux_x86_64.whl size=236769 sha256=76d5bc34bf24a054473ad84dc2bb4ee6c0deea75840a0f4265da5735fcce3c56
  Stored in directory: /root/.cache/pip/wheels/19/a0/e0/e04656d89dd723adbe

In [2]:
import os
import pickle
import numpy as np

from scipy.signal import find_peaks
from scipy import stats
import pyhrv.time_domain as td

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import kagglehub


In [3]:
root_path = kagglehub.dataset_download(
    "orvile/wesad-wearable-stress-affect-detection-dataset"
)

DATA_PATH = os.path.join(root_path, "WESAD")

print("DATA_PATH:", DATA_PATH)
print("Subjects:", os.listdir(DATA_PATH))


def load_subject(sid: str):
    p = os.path.join(DATA_PATH, f"S{sid}", f"S{sid}.pkl")
    with open(p, "rb") as f:
        data = pickle.load(f, encoding="latin1")

    chest = data["signal"]["chest"]

    ecg  = chest["ECG"].reshape(-1)
    resp = chest["Resp"].reshape(-1)
    acc  = chest["ACC"]          # (N,3)
    labels = data["label"].reshape(-1)

    return {
        "ecg": ecg,
        "resp": resp,
        "acc": acc,
        "label": labels,
    }

# sanity check
s2 = load_subject("2")
print("ECG:", s2["ecg"].shape)
print("RESP:", s2["resp"].shape)
print("ACC:", s2["acc"].shape)
print("LABEL:", s2["label"].shape)


Using Colab cache for faster access to the 'wesad-wearable-stress-affect-detection-dataset' dataset.
DATA_PATH: /kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD
Subjects: ['S14', 'S11', 'S13', 'S10', 'S8', 'S5', 'S7', 'S9', 'S15', 'wesad_readme.pdf', 'S2', 'S6', 'S3', 'S4', 'S16', 'S17']
ECG: (4255300,)
RESP: (4255300,)
ACC: (4255300, 3)
LABEL: (4255300,)


In [4]:
def extract_features(ecg_win, resp_win, acc_win, fs=700.0, win_seconds=10.0):
    # ACC magnitude
    acc_mag = np.linalg.norm(acc_win, axis=1)

    # ECG peaks → HR + HRV
    peaks, _ = find_peaks(ecg_win, distance=int(0.3 * fs))
    rr = np.diff(peaks) / fs

    mean_hr = 60.0 / np.mean(rr) if len(rr) > 1 else 0.0

    try:
        rmssd = td.rmssd(rr)['rmssd'] if len(rr) > 2 else 0.0
    except:
        rmssd = 0.0

    # Respiration rate (Hz)
    resp_peaks, _ = find_peaks(resp_win, distance=int(0.5 * fs))
    breathing_rate = len(resp_peaks) / win_seconds

    # ACC variance
    acc_var = float(np.var(acc_mag))

    return np.array([mean_hr, rmssd, breathing_rate, acc_var], dtype=np.float32)


In [5]:
def build_dataset_for_subject(sid: str, window_size=7000, step=3500):
    s = load_subject(sid)
    ecg, resp, acc, labels = s["ecg"], s["resp"], s["acc"], s["label"]

    Xs, ys = [], []

    for start in range(0, len(ecg) - window_size, step):
        end = start + window_size

        ecg_w  = ecg[start:end]
        resp_w = resp[start:end]
        acc_w  = acc[start:end]

        feats = extract_features(ecg_w, resp_w, acc_w)

        # Majority label in this window
        label_mode = stats.mode(labels[start:end], keepdims=False)[0]

        Xs.append(feats)
        ys.append(label_mode)

    return np.vstack(Xs), np.array(ys, dtype=int)


In [6]:
subjects = ["2","3","4","5","6","7","8","9","10","11","13","14","15","16","17"]

X_subjects = {}
y_subjects = {}

for sid in subjects:
    print(f"Processing S{sid}...")
    Xs, ys = build_dataset_for_subject(sid)
    X_subjects[sid] = Xs          # (n_windows, 4)
    y_subjects[sid] = ys          # (n_windows,)

# Convert labels to binary: stress = label 2
STRESS_LABELS = [2]

y_subjects_bin = {
    sid: np.isin(y_subjects[sid], STRESS_LABELS).astype(int)
    for sid in subjects
}


Processing S2...
Processing S3...
Processing S4...
Processing S5...
Processing S6...
Processing S7...
Processing S8...
Processing S9...
Processing S10...
Processing S11...
Processing S13...
Processing S14...
Processing S15...
Processing S16...
Processing S17...


In [7]:
# Concatenate all windows to compute scaling
X_all_raw = np.vstack([X_subjects[sid] for sid in subjects])

PHYS_MEAN = X_all_raw.mean(axis=0).astype(np.float32)
PHYS_STD  = X_all_raw.std(axis=0).astype(np.float32)

print("PHYS_MEAN:", PHYS_MEAN)
print("PHYS_STD :", PHYS_STD)

def phys_scale(X):
    return (X - PHYS_MEAN) / PHYS_STD


PHYS_MEAN: [1.53799377e+02 1.25880196e+02 1.87475598e+00 6.02732180e-04]
PHYS_STD : [1.3753248e+01 5.1713493e+01 7.9733528e-02 1.4178814e-02]


In [8]:
SEQ_LEN = 10      # number of windows per sequence
SEQ_STEP = 1      # stride between sequences

def build_sequences_for_subject(X_raw, y_bin, seq_len=SEQ_LEN, seq_step=SEQ_STEP):
    # Scale the features
    X = phys_scale(X_raw)

    X_seqs = []
    y_seqs = []

    n = len(X)
    for start in range(0, n - seq_len, seq_step):
        end = start + seq_len

        seq_feats = X[start:end]         # (seq_len, 4)
        seq_labels = y_bin[start:end]    # (seq_len,)

        # Label for the sequence = majority of window labels
        label_mode = stats.mode(seq_labels, keepdims=False)[0]

        X_seqs.append(seq_feats)
        y_seqs.append(label_mode)

    return np.stack(X_seqs), np.array(y_seqs, dtype=int)  # (n_seq, seq_len, 4), (n_seq,)


In [9]:
X_seq_all = []
y_seq_all = []

for sid in subjects:
    X_raw = X_subjects[sid]
    y_bin = y_subjects_bin[sid]

    Xs, ys = build_sequences_for_subject(X_raw, y_bin)
    X_seq_all.append(Xs)
    y_seq_all.append(ys)
    print(f"Subject S{sid}: sequences={Xs.shape[0]}")

X_seq_all = np.vstack(X_seq_all)  # (N_seq, SEQ_LEN, 4)
y_seq_all = np.concatenate(y_seq_all)

print("Final sequence dataset:", X_seq_all.shape, y_seq_all.shape)
print("Binary labels (sequence level):", np.unique(y_seq_all, return_counts=True))


Subject S2: sequences=1204
Subject S3: sequences=1287
Subject S4: sequences=1273
Subject S5: sequences=1240
Subject S6: sequences=1403
Subject S7: sequences=1036
Subject S8: sequences=1082
Subject S9: sequences=1033
Subject S10: sequences=1088
Subject S11: sequences=1035
Subject S13: sequences=1096
Subject S14: sequences=1098
Subject S15: sequences=1039
Subject S16: sequences=1115
Subject S17: sequences=1172
Final sequence dataset: (17201, 10, 4) (17201,)
Binary labels (sequence level): (array([0, 1]), array([15222,  1979]))


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_seq_all, y_seq_all,
    test_size=0.2,
    random_state=42,
    stratify=y_seq_all
)

print("Train seqs:", X_train.shape, y_train.shape)
print("Test  seqs:", X_test.shape, y_test.shape)


Train seqs: (13760, 10, 4) (13760,)
Test  seqs: (3441, 10, 4) (3441,)


In [11]:
class StressSeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


batch_size = 64

train_ds = StressSeqDataset(X_train, y_train)
test_ds  = StressSeqDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

class StressLSTM(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        out, (h_n, c_n) = self.lstm(x)
        # Take last hidden state: (batch, hidden_dim)
        h_last = h_n[-1]
        logits = self.fc(h_last)
        return logits

model = StressLSTM().to(device)


Using device: cuda


In [13]:
# Class weights at sequence level
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
print("Class weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)


Class weights: tensor([0.5650, 4.3462], device='cuda:0')


In [14]:
epochs = 150

for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for Xb, yb in train_loader:
        Xb = Xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * Xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    scheduler.step()

    train_loss = total_loss / total
    train_acc  = correct / total

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss={train_loss:.4f} | Train Acc={train_acc:.3f}")


Epoch 001 | Loss=0.4887 | Train Acc=0.772
Epoch 005 | Loss=0.4068 | Train Acc=0.788
Epoch 010 | Loss=0.3703 | Train Acc=0.801
Epoch 015 | Loss=0.3561 | Train Acc=0.794
Epoch 020 | Loss=0.3355 | Train Acc=0.801
Epoch 025 | Loss=0.3081 | Train Acc=0.810
Epoch 030 | Loss=0.2910 | Train Acc=0.821
Epoch 035 | Loss=0.2822 | Train Acc=0.828
Epoch 040 | Loss=0.2599 | Train Acc=0.838
Epoch 045 | Loss=0.2358 | Train Acc=0.856
Epoch 050 | Loss=0.2284 | Train Acc=0.859
Epoch 055 | Loss=0.2218 | Train Acc=0.863
Epoch 060 | Loss=0.2140 | Train Acc=0.866
Epoch 065 | Loss=0.2002 | Train Acc=0.875
Epoch 070 | Loss=0.1930 | Train Acc=0.881
Epoch 075 | Loss=0.1888 | Train Acc=0.884
Epoch 080 | Loss=0.1829 | Train Acc=0.888
Epoch 085 | Loss=0.1780 | Train Acc=0.890
Epoch 090 | Loss=0.1764 | Train Acc=0.892
Epoch 095 | Loss=0.1722 | Train Acc=0.894
Epoch 100 | Loss=0.1714 | Train Acc=0.898
Epoch 105 | Loss=0.1675 | Train Acc=0.897
Epoch 110 | Loss=0.1674 | Train Acc=0.898
Epoch 115 | Loss=0.1640 | Train Ac

In [15]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb = Xb.to(device)
        yb = yb.to(device)

        logits = model(Xb)
        preds = logits.argmax(dim=1)

        all_preds.append(preds.cpu().numpy())
        all_labels.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

print(classification_report(all_labels, all_preds, digits=3))
print(confusion_matrix(all_labels, all_preds))


              precision    recall  f1-score   support

           0      0.984     0.882     0.930      3045
           1      0.495     0.886     0.635       396

    accuracy                          0.883      3441
   macro avg      0.739     0.884     0.783      3441
weighted avg      0.927     0.883     0.896      3441

[[2687  358]
 [  45  351]]


In [16]:
def predict_sequence_stress(raw_seq_2d):
    """
    raw_seq_2d: np.array of shape (SEQ_LEN, 4)
                with raw [HR, RMSSD, breathing, acc_var]
    """
    # Scale with PHYS_MEAN/STD
    scaled = phys_scale(raw_seq_2d).astype(np.float32)
    x = torch.tensor(scaled.reshape(1, SEQ_LEN, 4), dtype=torch.float32).to(device)

    model.eval()
    with torch.no_grad():
        logits = model(x)
        prob = torch.softmax(logits, dim=1)[0,1].item()
        stress_percent = prob * 100.0
    return stress_percent


In [31]:
def rule_based_correction(hr, rmssd, breath, movement, lstm_stress):
    stress = float(lstm_stress)

    # 1️⃣ Sport / exercise
    if movement >= 0.8 and rmssd > 35:
        stress = min(stress, 15)
        return stress, 0

    # 2️⃣ Panic / crisis
    if hr > 120 and rmssd < 8 and movement < 0.4:
        stress = max(stress, 90)
        return stress, 3

    # 3️⃣ High mental stress
    if hr > 100 and rmssd < 20 and movement < 0.4:
        stress = max(stress, 70)
        return stress, 2

    # 4️⃣ Sleep / deep rest
    if hr < 60 and rmssd > 70 and movement < 0.15:
        stress = min(stress, 5)
        return stress, 0

    # 5️⃣ Light activity / walking / talking
    if movement > 0.5 and rmssd > 30:
        stress = min(stress, 35)
        return stress, 1

    # 6️⃣ Default: trust LSTM
    stress = max(0, min(stress, 100))

    # Map to level
    if stress < 20:
        return stress, 0
    elif stress < 40:
        return stress, 1
    elif stress < 70:
        return stress, 2
    else:
        return stress, 3


In [38]:
import torch
import numpy as np
import torch.nn.functional as F

def predict_single(hr, rmssd, breath, movement):
    """
    Run LSTM on a single physiological snapshot.
    Returns stress percentage (0–100).
    """

    # Build sequence
    raw_seq = np.tile([hr, rmssd, breath, movement], (SEQ_LEN, 1))

    # Normalize
    scaled = (raw_seq - PHYS_MEAN) / PHYS_STD

    # Tensor → SAME DEVICE AS MODEL
    x = torch.tensor(
        scaled,
        dtype=torch.float32,
        device=device   # 🔥 THIS FIXES THE ERROR
    ).unsqueeze(0)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)

    return probs[0, 1].item() * 100



In [34]:
def safe_predict(hr, rmssd, breath, movement):
    """
    Returns:
      stress_pct (float): 0–100
      level (int): 0–3
    """

    # --------------------------------------------------
    # 1️⃣ Get LSTM prediction
    # --------------------------------------------------
    lstm_stress = predict_single(hr, rmssd, breath, movement)  # 0–100

    # --------------------------------------------------
    # 2️⃣ If LSTM is untrained / collapsed → use heuristic base
    # --------------------------------------------------
    if lstm_stress < 5:   # VERY IMPORTANT LINE
        # Physiological baseline stress estimation
        base = 0

        # HR contribution
        if hr > 110: base += 35
        elif hr > 95: base += 25
        elif hr > 80: base += 10

        # HRV contribution
        if rmssd < 15: base += 35
        elif rmssd < 25: base += 20
        elif rmssd < 40: base += 10

        # Breathing contribution
        if breath > 0.6: base += 20
        elif breath > 0.4: base += 10

        # Clamp
        stress = min(base, 60)

    else:
        stress = lstm_stress

    # --------------------------------------------------
    # 3️⃣ Rule-based correction (your fixed version)
    # --------------------------------------------------

    # Sport / exercise
    if movement >= 0.8 and rmssd > 35:
        stress = min(stress, 15)
        return stress, 0

    # Panic
    if hr > 120 and rmssd < 8 and movement < 0.4:
        stress = max(stress, 90)
        return stress, 3

    # High mental stress
    if hr > 100 and rmssd < 20 and movement < 0.4:
        stress = max(stress, 70)
        return stress, 2

    # Sleep
    if hr < 60 and rmssd > 70 and movement < 0.15:
        stress = min(stress, 5)
        return stress, 0

    # Walking / light activity
    if movement > 0.5 and rmssd > 30:
        stress = min(stress, 35)
        return stress, 1

    # --------------------------------------------------
    # 4️⃣ Final level mapping
    # --------------------------------------------------
    if stress < 20:
        return stress, 0
    elif stress < 40:
        return stress, 1
    elif stress < 70:
        return stress, 2
    else:
        return stress, 3


In [19]:
i = 0  # any index in [0, len(X_test)-1]
raw_seq = X_test[i] * PHYS_STD + PHYS_MEAN  # back to raw
print("Stress %:", predict_sequence_stress(raw_seq))


Stress %: 0.001125711696658982


In [20]:
# ---- CALM SEQUENCE (SEQ_LEN x 4) ----
calm_seq = np.tile(
    np.array([72, 45, 0.25, 0.10]),   # HR, RMSSD, Breath, ACCvar
    (SEQ_LEN, 1)
)

print("Calm Stress %:", predict_sequence_stress(calm_seq))


Calm Stress %: 2.390024374631229e-13


In [21]:
# ---- STRESS SEQUENCE ----
stress_seq = np.tile(
    np.array([115, 10, 0.65, 1.20]),  # High HR, low RMSSD, fast breathing, movement
    (SEQ_LEN, 1)
)

print("Stress Stress %:", predict_sequence_stress(stress_seq))


Stress Stress %: 99.62318539619446


In [22]:
# ---- TRANSITION SEQUENCE (first half calm, second half stress) ----
half = SEQ_LEN // 2

transition_seq = np.vstack([
    np.tile(np.array([72, 45, 0.25, 0.10]), (half, 1)),     # calm half
    np.tile(np.array([110, 12, 0.60, 1.10]), (SEQ_LEN-half, 1))  # stress half
])

print("Transition Stress %:", predict_sequence_stress(transition_seq))


Transition Stress %: 0.1723736640997231


In [23]:
calm_feat = np.array([72, 45, 0.25, 0.10])
calm_seq = np.tile(calm_feat, (SEQ_LEN, 1))
print("LSTM Calm %:", predict_sequence_stress(calm_seq))

LSTM Calm %: 2.390024374631229e-13


In [24]:
stress_feat = np.array([115, 10, 0.65, 1.20])

stress_seq = np.tile(stress_feat, (SEQ_LEN, 1))
print("LSTM Stress %:", predict_sequence_stress(stress_seq))


LSTM Stress %: 99.62318539619446


In [25]:
half = SEQ_LEN // 2

transition_seq = np.vstack([
    np.tile(np.array([72, 45, 0.25, 0.10]), (half, 1)),
    np.tile(np.array([110, 12, 0.60, 1.10]), (SEQ_LEN-half, 1))
])
# LSTM sees whole history
print("LSTM Transition %:",
      predict_sequence_stress(transition_seq))


LSTM Transition %: 0.1723736640997231


In [26]:
recovery_seq = np.vstack([
    np.tile(np.array([110, 12, 0.60, 1.10]), (half, 1)),
    np.tile(np.array([75, 40, 0.30, 0.15]), (SEQ_LEN-half, 1))
])
print("LSTM Recovery %:", predict_sequence_stress(recovery_seq))

LSTM Recovery %: 3.295528048586141e-12


In [27]:
sport_feat = np.array([150, 120, 1.8, 0.9])

sport_seq = np.tile(sport_feat, (SEQ_LEN, 1))
print("LSTM Sport %:", predict_sequence_stress(sport_seq))


LSTM Sport %: 99.99489784240723


In [28]:
j = np.random.randint(0, len(X_test))  # for LSTM
raw_seq = X_test[j] * PHYS_STD + PHYS_MEAN
print(j)
print("\n--- LSTM REAL TEST ---")
print("True:", y_test[j])
print("LSTM %:", predict_sequence_stress(raw_seq))

811

--- LSTM REAL TEST ---
True: 0
LSTM %: 5.158036764091607e-21


In [29]:
# Test cases (ready to use)
test_cases = {
    "deep_sleep":    [55,  75,  0.20, 0.05],  # → 2%
    "calm":          [72,  45,  0.25, 0.10],  # → 0%
    "light_active":  [85,  35,  0.35, 0.20],  # → <25%
    "moderate_stress": [95, 22, 0.50, 0.30],  # → 40-60%
    "high_stress":   [110, 12,  0.60, 0.25],  # → 75%
    "panic":         [130, 7,   0.90, 0.15],  # → 95%
    "walking":       [105, 35,  0.70, 0.60],  # → 20-40%
    "sport":         [150, 120, 1.80, 0.90],  # → 5%
}

In [39]:
# Test all cases quickly
for name, vals in {
    "deep_sleep": [55, 75, 0.20, 0.05],
    "calm": [72, 45, 0.25, 0.10],
    "light_active": [85, 35, 0.35, 0.20],
    "moderate_stress": [95, 22, 0.50, 0.30],
    "high_stress": [110, 12, 0.60, 0.25],
    "panic": [130, 7, 0.90, 0.15],
    "walking": [105, 35, 0.70, 0.60],
    "sport": [150, 120, 1.80, 0.90]
}.items():
    pct, lvl = safe_predict(*vals)
    print(f"{name:18} → {pct:6.2f}% (Level {lvl})")

deep_sleep         →   0.00% (Level 0)
calm               →   0.00% (Level 0)
light_active       →  20.00% (Level 1)
moderate_stress    →  40.00% (Level 2)
high_stress        →  70.00% (Level 2)
panic              →  90.00% (Level 3)
walking            →  35.00% (Level 1)
sport              →  15.00% (Level 0)
